In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
!git clone https://github.com/cbtn-data-science-ml/ml_datasets.git

fatal: destination path 'ml_datasets' already exists and is not an empty directory.


In [3]:
%pwd

'/Users/barrios/Desktop/GitHub/tensorflow-professional-developer/13_create_a_pretrained_FE_model'

In [4]:
import os
os.chdir('ml_datasets')

In [5]:
%pwd

'/Users/barrios/Desktop/GitHub/tensorflow-professional-developer/13_create_a_pretrained_FE_model/ml_datasets'

In [6]:
%ls

README.md          food_10/           reduced_dataset/
cats_vs_dogs/      ramen_sushi/       waffles_or_nachos/


In [7]:
!pip install --upgrade tensorflow
!conda install tensorflow-hub -y

Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.



In [11]:
!pip install --upgrade tensorflow tensorflow-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.4 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.8.0
    Uninstalling tensorflow-hub-0.8.0:
      Successfully uninstalled tensorflow-hub-0.8.0


In [23]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = 'reduced_dataset'


In [24]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

In [25]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1/255.0
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'food_10/test',
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 800 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [30]:
def create_model_from_kaggle_tfhub(model_url, num_classes=10):

    # Load the feature extractor model from TensorFlow Hub
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             trainable=False,  # Freeze the underlying model
                                             input_shape=(224, 224, 3))  # This might vary based on the model

    # Create a Sequential model
    model = tf.keras.Sequential([
        feature_extractor_layer,
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

In [31]:
resnet_url = 'https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/feature-vector/1'

In [32]:
resnet_model = create_model_from_kaggle_tfhub(resnet_url, num_classes=10)

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x3734b6c50> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
resnet_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = resnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
https://colab.research.google.com/drive/1T3Sm6WHVReTN7EY8p894HQaUd3FaN_gL?usp=sharing